# Lab 10

# [Bike Sharing Demand](https://www.kaggle.com/c/bike-sharing-demand/data)

**Task**: You are provided hourly rental data spanning two years. For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period.

**Dataset Features:**

- *datetime* - hourly date + timestamm
- *season* -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
- *holiday* - whether the day is considered a holiday
- *workingday* - whether the day is neither a weekend nor holiday
- *weather* - 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
- *temp* - temperature in Celsius
- *atemp* - "feels like" temperature in Celsius
- *humidity* - relative humidity
- *windspeed* - wind speed
- *casual* - number of non-registered user rentals initiated
- *registered* - number of registered user rentals initiated
- *count* - number of total rentals

## Think About the Problem First

Try to generate hypothesis. For example, since we have `datetime` feature, we can extract the hour because bike demand may vary depending on the specific time of the day. Next, there might be a relation between the weekday and the demand. For example, in weekends, there might be less demand of bikes. Similary, rainy weather or high temperature might reduce the demand of bikes, and so on.

### Import Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso, ElasticNet
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error, mean_squared_log_error

### Load and Check Data

In [ ]:
train_data = pd.read_csv('./data/bike-sharing/train.csv')
test_data = pd.read_csv('./data/bike-sharing/test.csv')

In [ ]:
print('Train Shape: ', train_data.shape)
print('Test Shape: ', test_data.shape)

In [ ]:
train_data.sample(5)

In [ ]:
X = train_data.iloc[:, 0:9]
Y = train_data['count']

print('Train X Shape: ', X.shape)
print('Train Y Shape: ', Y.shape)
print('Test Shape: ', test_data.shape)

### Check for Missing Values

In [ ]:
train_data.isna().sum(axis=0)

## Exploratory Analysis

In [ ]:
sns.displot(Y, kde=True)

In [ ]:
sns.displot(np.log(Y), kde=True)

In [ ]:
sns.histplot(X.season, bins=4)

In [ ]:
sns.displot(X.temp, kde=True)

In [ ]:
sns.displot(X.atemp, kde=True)

In [ ]:
sns.displot(X.windspeed, kde=True)

In [ ]:
sns.displot(X.humidity, kde=True)

### Preprocessing & Feature Engineering with Pipeline

To learn more about `Pipeline`, you can try the following resources:

- [ML Data Pipelines with Custom Transformers in Python](https://towardsdatascience.com/custom-transformers-and-ml-data-pipelines-with-python-20ea2a7adb65)
- [How to Transform Target Variables for Regression in Python](https://machinelearningmastery.com/how-to-transform-target-variables-for-regression-with-scikit-learn/)
- [Using scikit-learn Pipelines and FeatureUnions](http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html)
- [Pre-Process Data like a Pro: Intro to Scikit-Learn Pipelines](https://towardsdatascience.com/clean-efficient-data-pipelines-with-pythons-sklearn-2472de04c0ea)

However, a complete understanding of data transformations can be found in [`sklearn`](https://scikit-learn.org/stable/data_transforms.html) documentation. I urge you all to visit this page.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class ProcessDateTime(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print('Transforming datetime...')
        
        x_copy = X.copy()
        x_copy['month'] = x_copy.datetime.apply(lambda x : calendar.month_name[datetime.strptime(x,"%Y-%m-%d %H:%M:%S").weekday()])
        x_copy['weekday'] = x_copy.datetime.apply(lambda x : calendar.day_name[datetime.strptime(x,"%Y-%m-%d %H:%M:%S").weekday()])
        x_copy['hour'] = x_copy.datetime.apply(lambda x : datetime.strptime(x,"%Y-%m-%d %H:%M:%S").hour)
        x_copy['minute'] = x_copy.datetime.apply(lambda x : datetime.strptime(x,"%Y-%m-%d %H:%M:%S").minute)
        x_copy = x_copy.drop(['datetime'], axis=1)
        
        return x_copy

In [ ]:
pipeline = Pipeline([
    ('datetime', ProcessDateTime())
])

pipeline.fit_transform(X)

In [ ]:
class ProcessSeasonWeather(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print('Transforming season and weather...')
        x_copy = X.copy()
        x_copy['season'] = x_copy['season'].map({
            1: 'Spring',
            2: 'Summer',
            3: 'Fall',
            4: 'Winter'
        })
        x_copy['weather'] = x_copy['weather'].map({
            1: "Clear+FewClouds+PartlyCloudy,PartlyCloudy",
            2: "Mist+Cloudy,Mist+BrokenClouds,Mist+FewClouds,Mist",
            3: "LightSnow,LightRain+Thunderstorm+ScatteredClouds,LightRain+ScatteredClouds",
            4: "HeavyRain+IcePallets+Thunderstorm+Mist,Snow+Fog" 
        })
        return x_copy

In [ ]:
pipeline = Pipeline([
    ('datetime', ProcessDateTime()),
    ('seasonweather', ProcessSeasonWeather())
])

In [ ]:
pipeline.fit_transform(X)

In [ ]:
class DummyEncoding(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print('Dummy encoding...')
        x_copy = X.copy()
        x_copy = pd.get_dummies(x_copy)
        return x_copy

    
class RemoveFeature(BaseEstimator, TransformerMixin):
    def __init__(self, features=[]):
        self._features = features
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print('Removing features...')
        x_copy = X.copy()
        for f in self._features:
            if f in x_copy.columns:
                x_copy = x_copy.drop([f], axis=1)
        return x_copy

In [ ]:
pipeline = Pipeline([
    ('datetime', ProcessDateTime()),
    ('seasonweather', ProcessSeasonWeather()),
    ('dummyencode', DummyEncoding()),
    ('removefeature', RemoveFeature(features=['windspeed']))
])

In [ ]:
pipeline.fit_transform(X)

In [ ]:
pipeline = Pipeline([
    ('datetime', ProcessDateTime()),
    ('seasonweather', ProcessSeasonWeather()),
    ('dummyencode', DummyEncoding()),
    ('removefeature', RemoveFeature(features=['windspeed'])),
    ('scaler', StandardScaler())
])

In [ ]:
pipeline.fit_transform(X)

**Why we did not fit test data like we did form train data?**

In [ ]:
pipeline = Pipeline([
    ('datetime', ProcessDateTime()),
    ('seasonweather', ProcessSeasonWeather()),
    ('dummyencode', DummyEncoding()),
    ('removefeature', RemoveFeature(['windspeed'])),
    ('scaler', MinMaxScaler())
])

pipeline.fit(X)
X = pipeline.transform(X)
X_test = pipeline.transform(test_data)

**Try using `StandardScaler` instead of `MinMaxScaler` and see what happens.**

In [ ]:
print(X.shape)
print(X_test.shape)

In [ ]:
pd.DataFrame(X)

## Modeling

In [ ]:
lr = LinearRegression()
sgd = SGDRegressor()
rr = Ridge()
ls = Lasso()
en = ElasticNet()

A model with some regularization typically performs better than a model without any regularization, so you should generally prefer Ridge Regression over plain Linear Regression.

Lasso Regression uses an $l_1$ penalty, which tends to push the weights down to exactly zero. This leads to sparse models, where all weights are zero except for the most important weights. This is a way to perform feature
selection automatically, which is good if you suspect that only a few features actually matter. When you are not
sure, you should prefer Ridge Regression.

Elastic Net is generally preferred over Lasso since Lasso may behave erratically in some cases (when several features are strongly correlated or when there are more features than training instances). However, it does add an extra hyperparameter to tune. If you want Lasso without the erratic behavior, you can just use Elastic Net with an l1_ratio close to 1.

In [ ]:
sklearn.metrics.SCORERS.keys()

### Hyperparamter Tuning Using GridSearchCV

![](./images/cv.png)

In [ ]:
cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=27)

grid_ridge_lasso = {
    'alpha': np.arange(0, 1, 0.05)
}

grid_elastic = {
    'alpha': np.arange(0, 1, 0.05),
    'l1_ratio': np.arange(0, 1, 0.05)
}

lr_score = cross_val_score(lr, X, np.log(Y+0.0001), cv=cv, scoring='neg_mean_squared_log_error')
sgd_score = cross_val_score(sgd, X, np.log(Y+0.0001), cv=cv, scoring='neg_mean_squared_log_error')

rr_search = GridSearchCV(rr, grid_ridge_lasso, cv=cv, scoring='neg_mean_squared_log_error')
rr_score = rr_search.fit(X, np.log(Y+0.0001))

ls_search = GridSearchCV(ls, grid_ridge_lasso, cv=cv, scoring='neg_mean_squared_log_error')
ls_score = ls_search.fit(X, np.log(Y+0.0001))

en_search = GridSearchCV(en, grid_elastic, cv=cv, scoring='neg_mean_squared_log_error')
en_score = en_search.fit(X, np.log(Y+0.0001))

In [ ]:
print(np.mean(lr_score))
print(np.mean(sgd_score))

print(rr_score.best_score_)
print(ls_score.best_score_)
print(en_score.best_score_)

In [ ]:
predictions = np.exp(rr_score.best_estimator_.predict(X_test))
predictions = predictions.astype('int')

In [ ]:
predictions

In [ ]:
pd.DataFrame({
    'datetime': test_data.datetime,
    'count': predictions
}).to_csv('./output/submission.csv', index=False)

**Kaggle Score:** 1.02292 <br/>
**Winning Score:** 0.3375

# Evaluation Metrics

![Metrics](./images/metrics.jpg)

**Let's start by defining some dummy labels and their corresponding predictions.**

In [ ]:
pred1 = np.array([1, 2, 3, 4])
label1 = np.array([2, 3, 2, 3])

**The most obvious and naive way to measure error is to simply calculate the difference between them and sum them up. However, simple difference between the predictions and the labels does not always show the actual picture. For example, does the error `0` below show the actual state of the model?**

In [ ]:
np.sum(pred1 - label1)

**The problem was that when we were adding the errors, negative and positive errors like `-5` and `5` were cancelling themselves. However, `-5` means our prediction was `5` units short to the actual value. In contrast, `5` means our prediction was `5` units up from the actual value. If we get a total of `0` error by adding themselves, it does not make sense. Taking absolute difference can solve this problem.**

In [ ]:
# Absolute difference
np.sum(np.abs(pred1 - label1))

**Now `-5` will convert into its absolute value `5` and we can add it to the other `5` to get a total error of `10`. Now it gives us the actual picture of the performance of our model. But we can not compare absolute error between two different sized datasets.**

In [ ]:
pred2 = np.array([1, 2, 3, 4, 1, 2])
label2 = np.array([2, 3, 2, 3, 2, 3])

In [ ]:
# Absolute difference
np.sum(np.abs(pred2 - label2))

**Here we just added first to data again at the end. Basically, this is the same prediction like the previous but with 2 extra data. But we are getting extra error. This is because we are adding the individual errors. So, the more data we have the more error will it generate, which is not fair. To compare between different sized datasets, we can simply take the mean.**

In [ ]:
# Mean absolute difference
print(mean_absolute_error(pred1, label1))
print(mean_absolute_error(pred2, label2))

**Another option to mitigate the problem of taking simple difference is to square the difference so that there will be no negative number to cancel out each other. This has another added advantage that it penalizes large errors more (Compare the difference between 2 and 4, and $2^2$ and $4^2$). Also, squared terms are good for calculating derivatives and thus useful in gradient descent.**

In [ ]:
pred3 = np.array([1, 2, 3, 4, 1, 2])
label3 = np.array([2, 3, 2, 7, 2, 3])

pred4 = np.array([1, 2, 3, 4, 1, 2])
label4 = np.array([2, 3, 2, 10, 2, 3])

pred5 = np.array([1, 2, 3, 4, 1, 2])
label5 = np.array([2, 3, 2, 13, 2, 3])

In [ ]:
# Mean absolute difference
print(mean_absolute_error(pred3, label3))
print(mean_absolute_error(pred4, label4))
print(mean_absolute_error(pred5, label5))

In [ ]:
# Mean squared difference
print(mean_squared_error(pred3, label3))
print(mean_squared_error(pred4, label4))
print(mean_squared_error(pred5, label5))

**Squaring error terms also squares the units. For example, if we have an error of, say, $\$15$ dollars, it will be $\$225$ $dollar^2$ after squaring. But there is nothing such as $dollar^2$. So, we take the root of the squared error to get back the original unit.**

In [ ]:
# Root mean square error
print(np.sqrt(mean_squared_error(pred3, label3)))
print(np.sqrt(mean_squared_error(pred4, label4)))
print(np.sqrt(mean_squared_error(pred5, label5)))

**Squaring and amplifying large errors are not what always expected. There can be contexts where you would want to ignore large mistakes.** 

In [ ]:
pred = np.array([60, 70, 80, 90])
label = np.array([57, 74, 79, 89])

In [ ]:
print(mean_squared_error(pred, label))

**Next, we have added an outlier, where the real label is `7`, but the prediction is `85`. As usual, `mean squared error` penalizes large mistakes more.**

In [ ]:
pred = np.array([60, 70, 80, 90, 85])
label = np.array([57, 74, 79, 89, 7])

In [ ]:
print(mean_squared_error(pred, label))

**Mean squared log error (MSLE) or root mean squared log error (RMSLE) can avoid this situation. MSLE is basically MSE with the exception that we are calculating difference between $log(prediction)$ and $log(label)$ instead of plain $prediction$ and $label$.**

In [ ]:
print(mean_squared_log_error(pred, label))